In [2]:
import yfinance as yf
from bs4 import BeautifulSoup
from lxml import etree
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import datetime
import json
import time
import math

### Gather all the SP500 ticker symbols through Wikipedia

In [ ]:
## sp500 stocks 

all_tickers_xpath = '//span[text() = "S&P 500 component stocks"]/following::tbody[1]/tr/td[1]/a'
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').content, 'html.parser')
dom = etree.HTML(str(soup))

# find all tickers
all_tickers = [i.text for i in dom.xpath(all_tickers_xpath)]

# look at the first 5 ticker symbols
print('The first 5 ticker symbols:')
print(all_tickers[:5])

The first 5 ticker symbols:
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


### Gather all stock market data for every ticker symbol on the SP500

In [ ]:
# yfinance is very convenient and allows us to do this in one function call
group_tickers = yf.download(' '.join(all_tickers), group_by='tickers')
group_tickers.to_csv('stock_ticker_data.csv')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 1925-12-20 -> 2024-11-25)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [ ]:
# some summary statistics for the ticker data
rows_cnt, col_cnt = np.shape(group_tickers)
print('Number of rows:', rows_cnt, 'Number of cols:', col_cnt)

# see the last bit of data for the group ticker symbols
group_tickers.tail()

Number of rows: 15834 Number of cols: 3018


Ticker             WEC                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2024-11-19   99.300003   99.300003   98.360001   99.050003   99.050003   
2024-11-20   98.790001   99.309998   98.639999   99.269997   99.269997   
2024-11-21   99.050003  101.000000   98.650002  100.959999  100.959999   
2024-11-22  101.160004  101.430000  100.570000  100.660004  100.660004   
2024-11-25  101.000000  101.589996  100.370003  101.349998  101.349998   

Ticker                        MTB                                      ...  \
Price          Volume        Open        High         Low       Close  ...   
Date                                                                   ...   
2024-11-19  1582600.0  211.710007  214.690002  211.210007  213.710007  ...   
2024-11-20  1934300.0  214.750000  214.750000  211.399994  213.520004  ...   
2024-11-21  1928800.0  213.630005  218.919998  212.979996  216.619995  ...   
2024-11-22  1304600.0  216.250000  221.520004  216.220001  221.119995  ...   
2024-11-25  2264847.0  221.500000  225.699997  221.524994  221.949997  ...   

Ticker             DFS                                            COO  \
Price              Low       Close   Adj Close     Volume        Open   
Date                                                                    
2024-11-19  169.449997  172.500000  171.801102  1225700.0   98.940002   
2024-11-20  171.759995  172.770004  172.070007  1968600.0   98.699997   
2024-11-21  172.490005  174.910004  174.910004  1904800.0   99.080002   
2024-11-22  174.779999  179.289993  179.289993  1063000.0   99.949997   
2024-11-25  180.500000  182.669998  182.669998  1841820.0  101.760002   

Ticker                                                                 
Price             High         Low       Close   Adj Close     Volume  
Date                                                                   
2024-11-19   99.900002   98.000000   99.050003   99.050003   887800.0  
2024-11-20   99.190002   97.980003   99.080002   99.080002   898900.0  
2024-11-21  100.040001   98.129997   99.910004   99.910004   992400.0  
2024-11-22  101.190002   99.500000  100.800003  100.800003  1140400.0  
2024-11-25  103.320000  101.500000  102.400002  102.400002  2559683.0  

[5 rows x 3018 columns]

### Gathering all required textual data (for sentiment/ market condition embeddings)

In [ ]:
# I am working on 5 years of data for all the tickers given by the SP500.

# literally all reddit data held in the Pushshift api dumps for years up to 2023:
#   you will need Torrent to deal with this data
# https://www.reddit.com/r/pushshift/comments/1akrhg3/separate_dump_files_for_the_top_40k_subreddits/


<h5>Reading CSV data of stock market and formatting</h5>

<h4>Unfortunately, 500 stock indicators is too much data, so we have to shorten this list to a smaller subset of stocks to study. We will focus on the top 25 stocks of SP500.</h4>

In [3]:
top_stocks = '''AAPL MSFT AMZN NVDA GOOGL BRK.A GOOG TSLA META XOM UNH JNJ JPM V PG MA CVX HD ABBV LLY MRK AVGO PEP KO PFE'''
top_stocks = top_stocks.split(' ')

# subset the dataset to the rows that contain the stock symbols
data_subset = data[data['Ticker'].isin(top_stocks)]

In [4]:
grouped = data_subset.groupby('Ticker')
# this example group will just be the sample dataset that
# we may do calculations with 
example_group = grouped.get_group(top_stocks[0])

In [5]:
# finanicial indicator functions
# note that these are indented to work with ONE key value
# ex. ONLY DATES means that we must group by stock ticker
# note: TICKERS will not make sense for these functions as keys

def check_series_numeric(column) -> bool:
    not_numeric = column.apply(pd.to_numeric, errors = 'coerce').isna().all()
    # if the data is not numeric, the test failed 
    return not not_numeric

def calculate_sma(column, period = 10):
    if not check_series_numeric(column):
        return column
    sma = column.rolling(period).mean()
    return pd.Series(sma, index= column.index, name = 'SMA')

def calculate_ema(column, period = 10):
    if not check_series_numeric(column):
        return column
    ema = column.ewm(span = period, min_periods = period - 1).mean()
    return pd.Series(ema, index = column.index, name = 'EMA')

# formula for RSI: 
'''
    let RS = average gain / average loss   
    RSI = 100 - 100 / (1 - RS)

    some things to look out for: 
        - using exponential weighted means on the average
            gains and losses made the RSI value much more
            sensitive and fluctuating more based on price
            changes
        - using simple averages made the RSI value more 
            smoothed out

    keyword argument options:
        - period: type int
        - use_exponential: type bool

'''
def calculate_rsi(column, *args, **kwargs):
    rsi = column.rolling(14).apply(lambda x: calculate_rsi_helper(x, *args, **kwargs)).astype(float)
    return pd.Series(rsi, index = column.index, name = 'RSI')

def calculate_rsi_helper(column, period = 14, use_exponential = False):
    if not check_series_numeric(column):
        return column 

    column = column.astype(float)
    # find differences in prices
    differences = column.diff() 

    # clip, but center off of zero value
    gain = differences.clip(lower = 0.01)
    loss = differences.clip(upper = -0.01)

    avg_gain = gain.mean()
    avg_loss = loss.mean()

    if use_exponential:
        # get the exponential weighted mean of the very last element in this current rolling window
        avg_gain = gain.ewm(span = period, min_periods = period - 1).mean()[-1]
        avg_loss = loss.ewm(span = period, min_periods = period - 1).mean()[-1]

    RS = avg_gain / avg_loss 
    RSI = 100 - 100 / (1 - RS)

    return RSI

'''
    MACD (moving average convergence/ divergence) shows the relationship
    between two exponential moving averages and comparing this to the 
    9-day EMA line 

    MACD = 12-period EMA - 26-period EMA 
    Signal = 9-period EMA - 26-period EMA
'''
def calculate_macd(column, long_period = 26, short_period = 12, signal_period = 9) -> tuple[object, object]:
    if not check_series_numeric(column):
        return column 
    
    ewm_short = example_group['Adj Close'].ewm(span = short_period, min_periods = 11).mean()
    ewm_long = example_group['Adj Close'].ewm(span = long_period, min_periods = 25).mean()

    # returns the macd 
    macd = (ewm_short - ewm_long).astype(float)

    # returns the signal line
    ewm_signal = example_group['Adj Close'].ewm(span = signal_period, min_periods = 8).mean()
    signal = (ewm_signal - ewm_long).astype(float)


    macd = pd.Series(macd, index = column.index, name = 'MACD')
    signal = pd.Series(signal, index = column.index, name = 'MACD-SIGNAL')
    return macd, signal


In [6]:
# TODO: apply this process concurrently or parallel if possible, then we can get
# even more data available to us
dataframes = []
for ticker in top_stocks:
    cur = None
    try:
        cur = grouped.get_group(ticker)
    except:
        print('skipped ticker:', ticker)
        continue 

    stock_data = cur['Adj Close']
    sma = calculate_sma(stock_data)
    ema = calculate_ema(stock_data)
    rsi = calculate_rsi(stock_data)
    macd, signal_macd = calculate_macd(stock_data)

    overall = pd.DataFrame([sma, ema, rsi, macd, signal_macd]).T
    res = pd.merge(cur, overall, on='Date')

    res = res.set_index([res.index, res['Ticker']]).drop('Ticker', axis = 1)

    dataframes.append(res)
    


skipped ticker: BRK.A


<h4>Transforming indicators to give us values that are a bit more useful</h4>

<ol>
    <li>Each entry of stocks are associated with the current day and the ticker symbol</li>
    <li>We may search through top comments or posts through Pushshift to search for stock ticker mentions per day and stock</li>
    <li>In order to load numerous http requests, we are using asyncio for concurrently loading data</li>
</ol>

In [240]:
# function to convert date time string into epoch format for pushshift
def str_index_to_epoch(s: str, days_after = 1) -> tuple[int, int]:
    s = str(s)
    s = s.split('-')
    yr = int(s[0])
    mo = int(s[1])
    day = int(s[2].split(' ')[0])
    # the yahoo finance data is given as EST time
    today = datetime.datetime(yr, mo, day)
    tomorrow = today + relativedelta(days = days_after)

    print('today', today, 'tomorrow', tomorrow, 'after', days_after)
    return (int(today.timestamp()), int(tomorrow.timestamp()))

def index_str_to_datetime(s: str, split='-'):
    # get anything that is before a space, to isolate the current search thingy
    s = s.split(' ')[0]
    vals = [int(i) for i in s.split(split)]

    return datetime.datetime(year=vals[0], month=vals[1], day=vals[2])

In [341]:
def get_link(X, subreddit = 'stocks', size = 25, days_after = 1):
    subreddit = subreddit
    ticker = X['Ticker']
    date_after, date_before = str_index_to_epoch(X.name, days_after=days_after)
    size = size

    req_string_reddit_comments = f'https://api.pushshift.io/reddit/search/comment/?q={ticker}&subreddit={subreddit}&size={size}&after={date_after}&before={date_before}&fields=body&frequency=day'
    
    return req_string_reddit_comments

def create_query_link(argument_dict, type = 'comment'):
    args = []
    for argument in argument_dict:
        args.append(argument)
        args.append('=')
        args.append(argument_dict[argument])
        args.append('&')

        query_string = ''.join(args[:-1])
    return f'https://api.pushshift.io/reddit/{type}/search/?{query_string}'


we can simply use the api to get all date ranges for the stock market in one link

In [371]:
# using ticker aapl as the current symbol
stock_data = data.query('Ticker == "AAPL"').reset_index().set_index('Date')
start_date, end_date = stock_data.iloc[0].name, stock_data.iloc[-1].name
start_date, end_date = index_str_to_datetime(str(start_date)), index_str_to_datetime(str(end_date))

period = (end_date - start_date).days
print(period, 'days to search for')

args = {
    'q': 'AAPL',
    'subreddit': 'stocks',
    'after': str(int(start_date.timestamp())),
    'before': str(int((end_date.timestamp()))),
    'size': str(500),
    'count': str(50),
    'sort':'created_utc',
    'sort_type': 'asc'
}

link = create_query_link(args)
print(link)

836 days to search for
https://api.pushshift.io/reddit/comment/search/?q=AAPL&subreddit=stocks&after=1609740000&before=1681966800&size=500&count=50&sort=created_utc&sort_type=asc


In [425]:
import threading 

dataframes = []

def roll_apply(window):
    print(window)

def add_to_dataframes(start, end, counter):
    after = index_str_to_datetime(str(start)).timestamp()
    before = index_str_to_datetime(str(end)).timestamp()

    args = {
    'q': 'AAPL',
    'subreddit': 'stocks',
    'after': str(int(after)),
    'before': str(int(before)),
    'size': str(500),
    'count': str(50),
    'sort':'created_utc',
    'sort_type': 'asc'
    }

    print('request #', counter)

    r = requests.get(create_query_link(args))
    result = json.loads(r.text)['data']
    dataframes.append(pd.DataFrame(result))

counter = 1
period = 30

cur_threads = []
for i in range(0, len(stock_data), period):
    start = stock_data.iloc[i].name
    end = stock_data.iloc[min(len(stock_data)-1,i + period)].name

    thread = threading.Thread(target=add_to_dataframes, args=(start, end, counter))
    thread.start()
    cur_threads.append(thread)

    time.sleep(2)

    counter += 1

for thread in cur_threads:
    thread.join()

request # 1
request # 2
request # 3
request # 4
request # 5
request # 6
request # 7
request # 8
request # 9
request # 10
request # 11
request # 12
request # 13
request # 14
request # 15
request # 16
request # 17
request # 18
request # 19
request # 20


In [426]:
sentiment_data = pd.concat(dataframes, axis = 0)
sentiment_data = sentiment_data[['body', 'utc_datetime_str']]
sentiment_data['date'] = [index_str_to_datetime(s) for s in sentiment_data['utc_datetime_str']]
sentiment_data['date'] = [convert_to_datetime(str(s)) for s in sentiment_data['date']]

sentiment_data

,body,utc_datetime_str,date
0,You're gonna wanna add PLTR by the EOY. Other...,2021-02-17 05:58:16,2021-02-17
1,I started a 1000$ (1087$ to be exact) trading ...,2021-02-17 05:56:23,2021-02-17
2,AAPL and MSFT are like bank accounts with 20-4...,2021-02-17 05:33:25,2021-02-17
3,Message unclear. YOLO life savings on AAPL at ...,2021-02-17 05:31:52,2021-02-17
4,Leaps on AAPL are pricey. If you’re going in ...,2021-02-17 05:27:41,2021-02-17
...,...,...,...
94,I don’t mean market consensus “undeniable winn...,2023-04-11 13:48:42,2023-04-11
95,I had 6680 shares of Waste Management in 2012 ...,2023-04-11 13:17:25,2023-04-11
96,"Portfolio 1: \nCSPX, VXUS \nAAPL, ABNB, AZN,...",2023-04-11 12:24:42,2023-04-11
97,AAPL. Strong buybacks and good dividend backed...,2023-04-11 11:21:45,2023-04-11
